<a href="https://colab.research.google.com/github/phdosilva/taes/blob/main/TAES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação da biblioteca do Z3 solver

In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 21.1 MB/s eta 0:00:00


# Função que busca conflitos nos requisitos convertidos para Lógica de 1ª ordem

Para eficiência computacional, serão checados subconjuntos de, no máximo, 3 requisitos.

In [2]:
from z3 import *
from itertools import combinations

def find_unsat_subsets(constraints):
    solver = Solver()
    solver.add(constraints)

    if solver.check() != unsat:
        print("The full set of constraints is satisfiable.")
        return

    print("The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...")

    unsat_subsets = []
    max_size = min(3, len(constraints))

    for i in range(1, max_size + 1):
        for subset in combinations(constraints, i):
            solver = Solver()
            solver.add(subset)
            if solver.check() == unsat:
                unsat_subsets.append(set(subset))

    for idx, subset in enumerate(unsat_subsets, 1):
        print(f"UNSAT subset {idx}:")
        for c in subset:
            print(c)
        print("---")

    return unsat_subsets

# Funcionamento

Para cada LLM, foi introduzido o seguinte prompt:

```
You are an virtual assistant, especialized in converting software requirements - which are expressed in natural language form - into first-order logic form.

Your task is to:
1. Read the entry, where each line is a requirement; and
2. Convert each requirement into its logic form;
3. Convert each logic form into z3-solver (python library) code.

Your returned message shall only contain the z3-solver code, without other explanations or process description.

The z3 constraints should be in a array named "constraints", so we only use the "s.add()" method once, to add all the constraints

Key points to consider while performing the task:
- Take into account other requirements to unify logic symbols;
- The requirements entry does not need addition of new requirements;
- Do not provide analysis on requirement contradiction. Your task is only to make the conversions as described above.

Entry:
"""
<Requisitos separados por uma nova linha>
"""
```

Após a obtenção do código, realizou-se:

1. Verificação de satisfatibilidade; e
1. Obtenção dos subconjuntos confiltantes, caso o conjunto tenha sido insatisfatível

# ChatGPT 4

In [6]:
from z3 import *

# Define variables
failed_logins = Int('failed_logins')
account_locked = Bool('account_locked')
email_sent = Bool('email_sent')
sms_sent = Bool('sms_sent')
invoice_deleted = Bool('invoice_deleted')
broker = Bool('broker')
subscriber = Bool('subscriber')
pbx_account = Bool('pbx_account')
virtual_circuit = Bool('virtual_circuit')
pbx_unpaid = Bool('pbx_unpaid')
rate_table_access = Bool('rate_table_access')
rate_table_permissions = Bool('rate_table_permissions')
admin = Bool('admin')
user_has_id = Bool('user_has_id')
custom_prefix = Bool('custom_prefix')
integration_payment_gateway = Bool('integration_payment_gateway')
payment_history = Bool('payment_history')
logging_enabled = Bool('logging_enabled')
analytics_admins_brokers = Bool('analytics_admins_brokers')
multi_currency = Bool('multi_currency')
backup_monthly = Bool('backup_monthly')
crm_erp_integration = Bool('crm_erp_integration')
bulk_invoice = Bool('bulk_invoice')
accounting_integration = Bool('accounting_integration')
broker_dashboard = Bool('broker_dashboard')
broker_commission = Bool('broker_commission')
credit_limit = Bool('credit_limit')
account_termination = Bool('account_termination')
account_suspension = Bool('account_suspension')
broker_registration_verified = Bool('broker_registration_verified')
modify_closed_account = Bool('modify_closed_account')
account_audit_monthly = Bool('account_audit_monthly')
account_audit_weekly = Bool('account_audit_weekly')
foreign_account_creation = Bool('foreign_account_creation')
foreign_account_active = Bool('foreign_account_active')
foreign_account_locked = Bool('foreign_account_locked')
foreign_account_closed = Bool('foreign_account_closed')
rate_table_print = Bool('rate_table_print')
modify_new_subscriptions = Bool('modify_new_subscriptions')
assign_custom_numbers = Bool('assign_custom_numbers')
overdue_highlight = Bool('overdue_highlight')
delete_subscriber_data = Bool('delete_subscriber_data')
overdue_highlight_configurable = Bool('overdue_highlight_configurable')
unique_subscription_number = Bool('unique_subscription_number')
invoice_flagged = Bool('invoice_flagged')
invoice_days = Int('invoice_days')
invoice_cleanup = Bool('invoice_cleanup')
view_closed_broker_accounts = Bool('view_closed_broker_accounts')
reopen_closed_accounts = Bool('reopen_closed_accounts')
payment_backup_dependency = Bool('payment_backup_dependency')
prevent_invoice_table_backup = Bool('prevent_invoice_table_backup')

s = Solver()

# Constraints
constraints = [
    Implies(failed_logins >= 3, account_locked),
    Implies(account_locked, email_sent),
    Implies(account_locked, Or(email_sent, sms_sent)),
    Not(invoice_deleted),
    Implies(broker, delete_subscriber_data),
    Implies(subscriber, Not(delete_subscriber_data)),
    Implies(pbx_account, Not(virtual_circuit)),
    Implies(pbx_unpaid, Not(virtual_circuit)),
    Implies(broker, rate_table_access),
    Implies(rate_table_permissions, admin),
    Implies(user_has_id, custom_prefix),
    Implies(Or(broker, subscriber), integration_payment_gateway),
    Implies(integration_payment_gateway, payment_history),
    logging_enabled,
    analytics_admins_brokers,
    Implies(multi_currency, True),
    backup_monthly,
    Implies(crm_erp_integration, True),
    Implies(broker, bulk_invoice),
    Implies(accounting_integration, True),
    Implies(broker, broker_dashboard),
    Implies(broker, broker_commission),
    Implies(broker, credit_limit),
    Implies(admin, account_termination),
    Implies(admin, account_suspension),
    Implies(broker_registration_verified, True),
    Implies(broker, Not(modify_closed_account)),
    account_audit_monthly,
    account_audit_weekly,
    Not(foreign_account_creation),
    Implies(Not(foreign_account_active), foreign_account_locked),
    Implies(foreign_account_locked, Not(foreign_account_active)),
    Implies(foreign_account_locked, foreign_account_closed),
    Implies(foreign_account_active, Not(foreign_account_closed)),
    Implies(broker, rate_table_print),
    Implies(broker, modify_new_subscriptions),
    Implies(broker, assign_custom_numbers),
    Implies(invoice_days > 60, invoice_flagged),
    Implies(invoice_flagged, invoice_cleanup),
    Implies(admin, view_closed_broker_accounts),
    Implies(view_closed_broker_accounts, reopen_closed_accounts),
    Implies(admin, Not(reopen_closed_accounts)),
    Implies(integration_payment_gateway, payment_backup_dependency),
    Implies(payment_backup_dependency, backup_monthly),
    Implies(integration_payment_gateway, prevent_invoice_table_backup),
    Implies(pbx_account, Not(virtual_circuit))
]

s.add(constraints)

In [7]:
s.check()

sat

In [8]:
find_unsat_subsets(constraints)

The full set of constraints is satisfiable.


# Deepseek R1

In [9]:
from z3 import *

# Declarations
failed_login_attempts = Int('failed_login_attempts')
account_access_allowed = Bool('account_access_allowed')
account_locked = Bool('account_locked')
notification_email_sent = Bool('notification_email_sent')
sms_sent = Bool('sms_sent')
invoice_annually = Bool('invoice_annually')
invoice_monthly = Bool('invoice_monthly')
invoice_deletion_allowed = Bool('invoice_deletion_allowed')
broker_delete_subscriber_allowed = Bool('broker_delete_subscriber_allowed')
invoices_stored_in_subscriber_accounts = Bool('invoices_stored_in_subscriber_accounts')
pbx_accounts_have_virtual_circuits = Bool('pbx_accounts_have_virtual_circuits')
deactivate_unpaid_pbx_circuits = Bool('deactivate_unpaid_pbx_circuits')
broker_access_rate_table = Bool('broker_access_rate_table')
admin_manage_rate_table_permissions = Bool('admin_manage_rate_table_permissions')
unique_user_ids = Bool('unique_user_ids')
customize_id_prefixes = Bool('customize_id_prefixes')
send_notifications = Bool('send_notifications')
integrate_payment_gateways = Bool('integrate_payment_gateways')
record_payment_history = Bool('record_payment_history')
log_user_actions = Bool('log_user_actions')
reporting_analytics = Bool('reporting_analytics')
multi_currency_support = Bool('multi_currency_support')
monthly_backups = Bool('monthly_backups')
crm_erp_integration = Bool('crm_erp_integration')
bulk_generate_invoices = Bool('bulk_generate_invoices')
accounting_software_integration = Bool('accounting_software_integration')
broker_dashboards = Bool('broker_dashboards')
calculate_broker_commissions = Bool('calculate_broker_commissions')
set_subscriber_credit_limits = Bool('set_subscriber_credit_limits')
admin_terminate_accounts = Bool('admin_terminate_accounts')
admin_suspend_accounts = Bool('admin_suspend_accounts')
new_broker_credential_verification = Bool('new_broker_credential_verification')
broker_modify_closed_account_data = Bool('broker_modify_closed_account_data')
monthly_account_audits = Bool('monthly_account_audits')
weekly_account_audits = Bool('weekly_account_audits')
foreign_account_creation_allowed = Bool('foreign_account_creation_allowed')
inactive_foreign_account_locked = Bool('inactive_foreign_account_locked')
broker_print_rate_tables = Bool('broker_print_rate_tables')
broker_modify_new_subscription_accounts = Bool('broker_modify_new_subscription_accounts')
assign_custom_numbers = Bool('assign_custom_numbers')
highlight_overdue_invoices = Bool('highlight_overdue_invoices')
broker_delete_subscriber_invoices = Bool('broker_delete_subscriber_invoices')
overdue_invoice_configurable = Bool('overdue_invoice_configurable')
auto_generate_subscription_numbers = Bool('auto_generate_subscription_numbers')
invoices_flagged_overdue = Bool('invoices_flagged_overdue')
cleanup_scheduling = Bool('cleanup_scheduling')
overdue_invoices_deletion_allowed = Bool('overdue_invoices_deletion_allowed')
admin_view_closed_broker_accounts = Bool('admin_view_closed_broker_accounts')
reopen_closed_accounts = Bool('reopen_closed_accounts')
admin_reopen_closed_accounts = Bool('admin_reopen_closed_accounts')
payment_gateway_record_history = Bool('payment_gateway_record_history')
payment_history_backups = Bool('payment_history_backups')
prevent_invoice_table_backups = Bool('prevent_invoice_table_backups')
locked_foreign_accounts_closed = Bool('locked_foreign_accounts_closed')
inactive_foreign_accounts_closed = Bool('inactive_foreign_accounts_closed')
pbx_virtual_circuits_allowed = Bool('pbx_virtual_circuits_allowed')

constraints = [
    # After 3 failed login attempts, the system must not allow account access.
    Implies(failed_login_attempts >= 3, Not(account_access_allowed)),

    # When locking an account, the system must send a notification email.
    Implies(account_locked, notification_email_sent),

    # When locking an account, the system must send an SMS unless email was sent.
    Implies(And(account_locked, Not(notification_email_sent)), sms_sent),

    # The system must invoice brokers annually using CDR.
    invoice_annually,

    # The system must invoice brokers monthly using CDR.
    invoice_monthly,

    # The system must not allow invoice deletion.
    Not(invoice_deletion_allowed),

    # Brokers must be allowed to delete subscriber accounts and data.
    broker_delete_subscriber_allowed,

    # Invoices must only be stored in subscriber accounts.
    invoices_stored_in_subscriber_accounts,

    # PBX accounts must not have virtual circuits.
    Not(pbx_accounts_have_virtual_circuits),

    # The system must deactivate unpaid PBX virtual circuits.
    deactivate_unpaid_pbx_circuits,

    # Brokers must be allowed to access the rate table.
    broker_access_rate_table,

    # Only administrators must access and manage rate table permissions.
    admin_manage_rate_table_permissions,

    # The system must assign unique user IDs.
    unique_user_ids,

    # Users must be allowed to customize ID prefixes.
    customize_id_prefixes,

    # The system must send invoice/approval/payment notifications to brokers and subscribers.
    send_notifications,

    # The system must integrate with payment gateways.
    integrate_payment_gateways,

    # Payment history must be recorded and accessible.
    record_payment_history,

    # The system must log all user actions (deletions/modifications/accesses).
    log_user_actions,

    # The system must provide reporting/analytics for admins and brokers.
    reporting_analytics,

    # The system must support multi-currency invoicing/payments when required.
    multi_currency_support,

    # Monthly backups and recovery plans must be maintained.
    monthly_backups,

    # CRM/ERP integration must be available when required.
    crm_erp_integration,

    # Brokers must be allowed to bulk-generate invoices.
    bulk_generate_invoices,

    # Accounting software integration must be available.
    accounting_software_integration,

    # Brokers must have dashboards showing account status/earnings.
    broker_dashboards,

    # The system must calculate broker commissions per paid invoice.
    calculate_broker_commissions,

    # Brokers must be allowed to set subscriber credit limits.
    set_subscriber_credit_limits,

    # Admins must be allowed to terminate accounts for violations.
    admin_terminate_accounts,

    # Admins must be allowed to temporarily suspend accounts.
    admin_suspend_accounts,

    # New broker registrations must require credential verification.
    new_broker_credential_verification,

    # Brokers must not modify closed account data.
    Not(broker_modify_closed_account_data),

    # Monthly account audits must be performed.
    monthly_account_audits,

    # Weekly account audits must be performed.
    weekly_account_audits,

    # Foreign account creation must not be allowed.
    Not(foreign_account_creation_allowed),

    # Inactive foreign accounts (>3 months) must be locked.
    inactive_foreign_account_locked,

    # Brokers must be allowed to print rate tables.
    broker_print_rate_tables,

    # Brokers must be allowed to modify new subscription accounts.
    broker_modify_new_subscription_accounts,

    # Brokers must be allowed to assign custom numbers to new accounts.
    assign_custom_numbers,

    # Overdue invoices must be highlighted in dashboards.
    highlight_overdue_invoices,

    # Brokers must be allowed to delete subscriber accounts/invoices.
    broker_delete_subscriber_invoices,

    # Overdue invoice highlighting must be configurable.
    overdue_invoice_configurable,

    # The system must auto-generate unique subscription numbers.
    auto_generate_subscription_numbers,

    # Brokers must be allowed to modify closed account data.
    broker_modify_closed_account_data,

    # Invoices >60 days old must be flagged as overdue.
    invoices_flagged_overdue,

    # Flagged invoices must enter cleanup scheduling.
    Implies(invoices_flagged_overdue, cleanup_scheduling),

    # Overdue invoices must not be deleted.
    Not(overdue_invoices_deletion_allowed),

    # Admins must be allowed to view closed broker accounts.
    admin_view_closed_broker_accounts,

    # Users with closed-account view access must be allowed to reopen them.
    reopen_closed_accounts,

    # Admins must not be allowed to reopen closed accounts.
    Not(admin_reopen_closed_accounts),

    # Payment Gateway integration must record payment history.
    payment_gateway_record_history,

    # When payment history is recorded, monthly backups must occur.
    Implies(record_payment_history, monthly_backups),

    # Payment Gateway integration must prevent monthly invoice table backups.
    prevent_invoice_table_backups,

    # Inactive foreign accounts must be locked.
    inactive_foreign_account_locked,

    # Locked foreign accounts must be closed after 30 days.
    locked_foreign_accounts_closed,

    # Inactive foreign accounts must not be closed.
    Not(inactive_foreign_accounts_closed),

    # The system must not allow PBX virtual circuits.
    Not(pbx_virtual_circuits_allowed),

    # The system must deactivate unpaid PBX circuits.
    deactivate_unpaid_pbx_circuits
]

s = Solver()
s.add(constraints)

In [10]:
s.check()

unsat

In [11]:
find_unsat_subsets(constraints)

The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...
UNSAT subset 1:
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 2:
Implies(failed_login_attempts >= 3,
        Not(account_access_allowed))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 3:
broker_modify_closed_account_data
Implies(account_locked, notification_email_sent)
Not(broker_modify_closed_account_data)
---
UNSAT subset 4:
Implies(And(account_locked, Not(notification_email_sent)),
        sms_sent)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 5:
broker_modify_closed_account_data
invoice_annually
Not(broker_modify_closed_account_data)
---
UNSAT subset 6:
invoice_monthly
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 7:
Not(invoice_deletion_allowed)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)


[{Not(broker_modify_closed_account_data), broker_modify_closed_account_data},
 {Implies(failed_login_attempts >= 3,
          Not(account_access_allowed)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, notification_email_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(And(account_locked, Not(notification_email_sent)),
          sms_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_annually},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_monthly},
 {Not(broker_modify_closed_account_data),
  Not(invoice_deletion_allowed),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_delete_subscriber_allowed,
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),


# Google Gemini

In [13]:
from z3 import *

s = Solver()

failed_login_attempts = Int('failed_login_attempts')
account_access = Bool('account_access')
account_locked = Bool('account_locked')
email_sent = Bool('email_sent')
sms_sent = Bool('sms_sent')
invoice_brokers_annually = Bool('invoice_brokers_annually')
invoice_brokers_monthly = Bool('invoice_brokers_monthly')
invoice_deletion_allowed = Bool('invoice_deletion_allowed')
broker_delete_subscriber_accounts = Bool('broker_delete_subscriber_accounts')
invoices_stored_subscriber_accounts = Bool('invoices_stored_subscriber_accounts')
pbx_virtual_circuits = Bool('pbx_virtual_circuits')
deactivate_unpaid_pbx_circuits = Bool('deactivate_unpaid_pbx_circuits')
broker_access_rate_table = Bool('broker_access_rate_table')
admin_access_manage_rate_table = Bool('admin_access_manage_rate_table')
unique_user_ids = Bool('unique_user_ids')
user_customize_id_prefixes = Bool('user_customize_id_prefixes')
send_notifications_brokers_subscribers = Bool('send_notifications_brokers_subscribers')
integrate_payment_gateways = Bool('integrate_payment_gateways')
payment_history_recorded_accessible = Bool('payment_history_recorded_accessible')
log_user_actions = Bool('log_user_actions')
reporting_analytics_admins_brokers = Bool('reporting_analytics_admins_brokers')
multi_currency_invoicing_payments = Bool('multi_currency_invoicing_payments')
monthly_backups_recovery = Bool('monthly_backups_recovery')
crm_erp_integration = Bool('crm_erp_integration')
bulk_generate_invoices = Bool('bulk_generate_invoices')
accounting_software_integration = Bool('accounting_software_integration')
broker_dashboards_account_status_earnings = Bool('broker_dashboards_account_status_earnings')
calculate_broker_commissions = Bool('calculate_broker_commissions')
broker_credit_limits = Bool('broker_credit_limits')
admin_terminate_accounts_violations = Bool('admin_terminate_accounts_violations')
admin_suspend_accounts = Bool('admin_suspend_accounts')
credential_verification = Bool('credential_verification')
broker_modify_closed_account_data = Bool('broker_modify_closed_account_data')
monthly_account_audits = Bool('monthly_account_audits')
weekly_account_audits = Bool('weekly_account_audits')
foreign_account_creation = Bool('foreign_account_creation')
inactive_foreign_accounts = Bool('inactive_foreign_accounts')
broker_print_rate_tables = Bool('broker_print_rate_tables')
broker_modify_new_subscription_accounts = Bool('broker_modify_new_subscription_accounts')
broker_assign_custom_numbers = Bool('broker_assign_custom_numbers')
overdue_invoices_highlighted = Bool('overdue_invoices_highlighted')
broker_delete_subscriber_invoices = Bool('broker_delete_subscriber_invoices')
overdue_highlighting_configurable = Bool('overdue_highlighting_configurable')
auto_generate_subscription_numbers = Bool('auto_generate_subscription_numbers')
invoices_over_60_days_overdue = Bool('invoices_over_60_days_overdue')
flagged_invoices_cleanup = Bool('flagged_invoices_cleanup')
overdue_invoices_deleted = Bool('overdue_invoices_deleted')
admin_view_closed_broker_accounts = Bool('admin_view_closed_broker_accounts')
user_reopen_closed_accounts = Bool('user_reopen_closed_accounts')
payment_history_recorded = Bool('payment_history_recorded')
monthly_invoice_table_backups = Bool('monthly_invoice_table_backups')
locked_foreign_accounts = Bool('locked_foreign_accounts')

constraints = [
    Implies(failed_login_attempts > 3, Not(account_access)),
    Implies(account_locked, email_sent),
    Implies(account_locked, Or(sms_sent, email_sent)),
    invoice_brokers_annually,
    invoice_brokers_monthly,
    Not(invoice_deletion_allowed),
    broker_delete_subscriber_accounts,
    invoices_stored_subscriber_accounts,
    Not(pbx_virtual_circuits),
    deactivate_unpaid_pbx_circuits,
    broker_access_rate_table,
    admin_access_manage_rate_table,
    unique_user_ids,
    user_customize_id_prefixes,
    send_notifications_brokers_subscribers,
    integrate_payment_gateways,
    payment_history_recorded_accessible,
    log_user_actions,
    reporting_analytics_admins_brokers,
    multi_currency_invoicing_payments,
    monthly_backups_recovery,
    crm_erp_integration,
    bulk_generate_invoices,
    accounting_software_integration,
    broker_dashboards_account_status_earnings,
    calculate_broker_commissions,
    broker_credit_limits,
    admin_terminate_accounts_violations,
    admin_suspend_accounts,
    credential_verification,
    Not(broker_modify_closed_account_data),
    monthly_account_audits,
    weekly_account_audits,
    Not(foreign_account_creation),
    Implies(inactive_foreign_accounts, account_locked),
    broker_print_rate_tables,
    broker_modify_new_subscription_accounts,
    broker_assign_custom_numbers,
    overdue_invoices_highlighted,
    broker_delete_subscriber_invoices,
    overdue_highlighting_configurable,
    auto_generate_subscription_numbers,
    broker_modify_closed_account_data,
    Implies(invoices_over_60_days_overdue, And(overdue_invoices_highlighted, invoices_over_60_days_overdue)),
    Implies(invoices_over_60_days_overdue, flagged_invoices_cleanup),
    Not(overdue_invoices_deleted),
    admin_view_closed_broker_accounts,
    Implies(admin_view_closed_broker_accounts, user_reopen_closed_accounts),
    Not(And(admin_view_closed_broker_accounts, user_reopen_closed_accounts)),
    Implies(integrate_payment_gateways, payment_history_recorded),
    Implies(payment_history_recorded, monthly_backups_recovery),
    Implies(integrate_payment_gateways, Not(monthly_invoice_table_backups)),
    Implies(inactive_foreign_accounts, account_locked),
    Implies(locked_foreign_accounts, account_locked),
    Implies(locked_foreign_accounts, And(locked_foreign_accounts, account_locked)),
    Not(And(inactive_foreign_accounts, locked_foreign_accounts)),
    Not(pbx_virtual_circuits),
    deactivate_unpaid_pbx_circuits,
]

s.add(constraints)

In [14]:
s.check()

unsat

In [15]:
find_unsat_subsets(constraints)

The full set of constraints is unsatisfiable. Finding conflicting subsets of max size 3...
UNSAT subset 1:
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 2:
Implies(failed_login_attempts > 3, Not(account_access))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 3:
Implies(account_locked, email_sent)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 4:
Implies(account_locked, Or(sms_sent, email_sent))
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 5:
invoice_brokers_annually
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 6:
Not(broker_modify_closed_account_data)
broker_modify_closed_account_data
invoice_brokers_monthly
---
UNSAT subset 7:
Not(invoice_deletion_allowed)
broker_modify_closed_account_data
Not(broker_modify_closed_account_data)
---
UNSAT subset 8:
broker_delete_subscri

[{Not(broker_modify_closed_account_data), broker_modify_closed_account_data},
 {Implies(failed_login_attempts > 3, Not(account_access)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, email_sent),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Implies(account_locked, Or(sms_sent, email_sent)),
  Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_brokers_annually},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoice_brokers_monthly},
 {Not(broker_modify_closed_account_data),
  Not(invoice_deletion_allowed),
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_delete_subscriber_accounts,
  broker_modify_closed_account_data},
 {Not(broker_modify_closed_account_data),
  broker_modify_closed_account_data,
  invoi